In [1]:
import bs4
from bs4 import BeautifulSoup
import requests
import pandas
from pandas import DataFrame
import csv

In [2]:
with open('iproperty.csv', mode='w') as csv_file:
    fieldnames = ['link', 
                  'name', 
                  'type', 
                  'price', 
                  'address', 
                  'built_up', 
                  'land_area', 
                  'bedrooms', 
                  'bathrooms', 
                  'monthly_installment', 
                  'land_title', 
                  'tenure',
                  'price_per_sqft',
                  'maintenance_fee',
                  #'near_by_amenities',
                  #'facilities',
                  'furnishing',
                  #'floor_level',
                  'state']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

In [3]:
property_link = []
property_name = []
property_type = []
property_price = []
property_address = []
property_built_up = []
property_land_area = []
property_bedrooms = []
property_bathrooms = []
property_monthly_installment = []
property_land_title = []
property_tenure = []
property_price_per_sqft = []
property_maintenance_fee = []
#property_near_by_amenities = []
#property_facilities = []
property_furnishing = []
#property_floor_level = []
property_state = []

In [4]:
def scraping(webpage, page_number):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    next_page = webpage + str(page_number)
    response= requests.get(str(next_page), headers=headers)
    soup = BeautifulSoup(response.text,"html.parser")
    soup_link = soup.findAll("div", attrs={"class":'PremiumCardstyle__BottomDescriptionWrapper-iGyVff czlWoj'})
    soup_type = soup.findAll("p",{"class":"attributes-description-item ListingAttributesstyle__ListingAttrsDescriptionItemWrapper-eRPpTH gjIvWo"})
    soup_state = soup.findAll("div",{"class":"PremiumCardstyle__AddressWrapper-jSJtsE hlWmFZ"})
    
    for x in range(len(soup_link)):
        pty_link = soup_link[x].find('a', { "class": "depth-listing-card-link" })
        if pty_link.text=="View details":
            property_link.append(pty_link['href'])
            print(pty_link['href'])
        property_type.append(soup_type[x].text.split('•')[0])
        property_state.append(soup_state[x].text.split(',')[1])
        
        if pty_link.text=="View details":
            detail_page = "https://www.iproperty.com.my" + pty_link['href']
            response= requests.get(str(detail_page))
            soup = BeautifulSoup(response.content,"html.parser")
            
            soup_name = soup.find("h1",attrs={"class":'PropertySummarystyle__ProjectTitleWrapper-cvJoqY ifcFYS'})
            property_name.append(soup_name.text)
            
            soup_price = soup.find("div", attrs={"class":'property-price ListingPrice__Price-cmdnCI ieWmRM'})
            if 'Contact' in soup_price.text or 'From' in soup_price.text:
                property_price.append(0)
            else:
                property_price.append(soup_price.text.split('RM')[1].replace(',', ''))
                
            soup_address_1 = soup.findAll("span", {"class":"property-address sale-default"})
            soup_address_2 = soup.findAll("span", {"class":"property-address new-ppp"})

            try:
                property_address.append(soup_address_1[0].text)
            except IndexError:
                property_address.append(soup_address_2[0].text)
            
            soup_built_up_land_area = soup.find("div",attrs={"class":'property-areas-info PropertySummarystyle__AreaInfo-iWQlgB Brfso'})
            
            soup_built_up = soup_built_up_land_area.find("li", attrs={"class":"PropertySummarystyle__AreaInfoItem-lmykjH kdRqkQ"})
            property_built_up.append(soup_built_up.text.split(':')[1].replace(',', ''))
            
            soup_land_area = soup_built_up.find_next_sibling("li")
            property_land_area.append(soup_land_area.text.split(':')[1].replace(',', ''))
            
            soup_bedrooms_bathrooms = soup.find("ul", {"class":"property-features PropertyFeatures__PropertyFeaturesWrapper-fAoHZZ WBFTH"})
            
            try:
                if soup_bedrooms_bathrooms==None:
                    property_bedrooms.append("0")
                else:
                    property_bedrooms.append(soup_bedrooms_bathrooms.findAll("li")[0].text)
            except IndexError:
                if soup_bedrooms_bathrooms==None:
                    property_bedrooms.append("0")
                else:
                    property_bedrooms.append(soup_bedrooms_bathrooms.findAll("li")[0].text)

            try:
                if soup_bedrooms_bathrooms==None:
                    property_bathrooms.append("0")
                else:
                    try:
                        property_bathrooms.append(soup_bedrooms_bathrooms.findAll("li")[1].text)  
                    except IndexError:
                        property_bathrooms.append("0")
            except IndexError:
                if soup_bedrooms_bathrooms==None:
                    property_bathrooms.append("0")
                else:
                    property_bathrooms.append(soup_bedrooms_bathrooms.findAll("li")[1].text) 
                            
            soup_monthly_installment = soup.find("div", {"class":"money-per-month MortgageCalculatorstyle__MoneyWrapper-iRttZT haasVb"})
            property_monthly_installment.append(soup_monthly_installment.text.split('RM')[1].replace('per month', ''))
            
            soup_property_features = soup.findAll("div", {"class":"ListingAttributesstyle__AttributeItemData-dogytt dCAMnf"})
            soup_new_property_features = soup.findAll("div", {"class":"ListingAttributesstyle__AttributeItemData-dogytt hfXwpL"})
            
            try:
                property_land_title.append(soup_property_features[2].text)
            except IndexError:
                property_land_title.append(soup_new_property_features[0].text)     
                
            try:
                property_tenure.append(soup_property_features[4].text)
            except IndexError:
                property_tenure.append("-")
                        
            try:
                property_price_per_sqft.append(soup_property_features[8].text)
            except IndexError:
                property_price_per_sqft.append(soup_new_property_features[1].text)
            
            try:
                property_maintenance_fee.append(soup_property_features[10].text)
            except IndexError:
                property_maintenance_fee.append("-")
                
            #soup_facilities = soup.find("div", {"class":"feature-description-content ListingFeatureDescription__FeatureDescriptionContent-dFVvWS hPahWk"})
            #property_facilities.append(soup_facilities.text)
            
            try:
                property_furnishing.append(soup_property_features[6].text)
            except IndexError:
                property_furnishing.append("-")
        
    if page_number < 100:
        page_number = page_number + 1
        scraping(webpage, page_number)

In [ ]:
scraping('https://www.iproperty.com.my/sale/all-residential/?page=', 1)

/property/tropicana/sale-8250210/
/property/desa-park-city/sale-7179338/
/property/mont-kiara/arcoris-residences/sale-8478535/
/new-property/property/johor-bahru/harp-taman-desa-tebrau-2-storey-cluster-homes/new-5690/
/property/petaling-jaya/sale-7880015/
/property/damansara-perdana/sale-7062522/
/new-property/property/relau/setia-sky-vista/new-2015/
/property/desa-park-city/sale-5338505/
/property/sungai-ara/sale-8449869/
/property/petaling-jaya/sale-7420635/
/new-property/property/kulai/taman-saujana-jaya/new-6072/
/property/subang-jaya/isola/sale-5217829/
/property/bukit-jalil/the-park-sky-residence/sale-8444095/
/property/tropicana/sale-7748355/
/new-property/property/iskandar-puteri-nusajaya/aspira-parkhomes/new-5852/
/property/bukit-jalil/the-park-sky-residence/sale-8444069/
/property/tropicana/tropicana-avenue/sale-8332192/
/property/damansara-heights/sale-7192819/
/property/iskandar-puteri-nusajaya/sale-8156913/
/property/usj/sale-8256933/
/property/subang-jaya/sale-4288497/
/p

/new-property/property/dengkil/kita-harmoni/new-6024/
/property/ipoh/sale-7329255/
/property/putrajaya/sale-6332760/
/property/johor-bahru/sale-7191035/
/new-property/property/dengkil/kita-impian/new-6023/
/property/cyberjaya/lake-view-residency/sale-6677829/
/property/cyberjaya/sale-8128241/
/property/ampang/sale-7690097/
/new-property/property/bandar-puncak-alam/melodi-perdana/new-6043/
/property/cyberjaya/sale-8127381/
/property/cyberjaya/sale-8121716/
/property/cyberjaya/sale-7049828/
/property/tanjong-tokong/the-tamarind/sale-7364765/
/property/tampoi/sky-peak-residences/sale-8409171/
/property/bukit-jalil/the-tropika/sale-8465067/
/property/johor-bahru/sale-8286548/
/property/johor-bahru/sale-7336327/
/property/johor-bahru/sale-8321108/
/property/johor-bahru/sale-7336327/
/property/johor-bahru/sale-8321108/
/property/johor-bahru/sale-7085467/
/new-property/property/georgetown/setia-v-residences/new-1878/
/property/johor-bahru/sale-7068767/
/property/johor-bahru/sale-7068725/
/new

/property/selayang/amara/sale-8476934/
/property/oug/sale-8427690/
/property/seri-kembangan/sale-8262409/
/property/puchong/sale-8182925/
/property/rawang/sale-4362741/
/property/desa-park-city/sale-7440369/
/property/iskandar-puteri-nusajaya/sale-8476776/
/property/skudai/sale-8476721/
/new-property/property/seremban/lunaria-3-resort-homes-bandar-sri-sendayan/new-5639/
/property/kota-kemuning/sale-8476738/
/property/johor-bahru/sale-8467189/
/new-property/property/pengerang/precint-1-the-maris-desaru-coast/new-5875/
/property/iskandar-puteri-nusajaya/one-sentral-serviced-residence/sale-8090676/
/property/cyberjaya/aspire-residence/sale-7098518/
/property/johor-bahru/sale-7284255/
/new-property/property/pasir-gudang/the-orchid/new-5998/
/property/seremban/sale-6777090/
/property/jalan-klang-lama-old-klang-road/the-scott-garden-soho/sale-6710634/
/property/senawang/sale-6724923/
/new-property/property/cheras/m-vertica/new-5466/
/property/setia-tropika/sale-8466253/
/property/setia-tropi

In [ ]:
data = { 
    'link': property_link,
    'name':property_name, 
    'type':property_type, 
    'price':property_price, 
    'address':property_address,
    'built_up':property_built_up,
    'land_area':property_land_area,
    'bedrooms':property_bedrooms,
    'bathrooms':property_bathrooms,
    'monthly_installment':property_monthly_installment,
    'land_title':property_land_title,
    'tenure':property_tenure,
    'price_per_sqft':property_price_per_sqft,
    'maintenance_fee':property_maintenance_fee,
    #'near_by_amenities':property_near_by_amenities,
    #'facilities':property_facilities,
    'furnishing':property_furnishing,
    #'floor_level':property_floor_level,
    'state':property_state
}

In [ ]:
print(data)

In [ ]:
print ([len(v) for v in data.values()])

In [ ]:
df = DataFrame(data, columns = ['link', 
                  'name', 
                  'type', 
                  'price', 
                  'address', 
                  'built_up', 
                  'land_area', 
                  'bedrooms', 
                  'bathrooms', 
                  'monthly_installment', 
                  'land_title', 
                  'tenure',
                  'price_per_sqft',
                  'maintenance_fee',
                  #'near_by_amenities',
                  #'facilities',
                  'furnishing',
                  #'floor_level',
                  'state'])
df.to_csv(r'/Users/zero/Documents/GitHub/deep_property/iproperty.csv')